In [ ]:
import pandas as pd
from flaml import AutoML

# sklearn
from sklearn.model_selection import StratifiedGroupKFold

# my own modules
from scripts.pyslavseq.model_selection import SampleChrSplitter, Model, parse_log

(raylet) [2023-08-21 22:34:59,889 E 27728 27751] (raylet) file_system_monitor.cc:111: /iblm/netapp/data4/mcuoco/tmp/ray/session_2023-08-21_17-59-51_808372_27477 is over 95% full, available space: 3934035574784; capacity: 89830100500480. Object creation will fail if spilling is required.
(raylet) [2023-08-21 22:35:09,952 E 27728 27751] (raylet) file_system_monitor.cc:111: /iblm/netapp/data4/mcuoco/tmp/ray/session_2023-08-21_17-59-51_808372_27477 is over 95% full, available space: 3933501128704; capacity: 89830100500480. Object creation will fail if spilling is required.
(raylet) [2023-08-21 22:35:20,017 E 27728 27751] (raylet) file_system_monitor.cc:111: /iblm/netapp/data4/mcuoco/tmp/ray/session_2023-08-21_17-59-51_808372_27477 is over 95% full, available space: 3933136879616; capacity: 89830100500480. Object creation will fail if spilling is required.
(raylet) [2023-08-21 22:35:30,084 E 27728 27751] (raylet) file_system_monitor.cc:111: /iblm/netapp/data4/mcuoco/tmp/ray/session_2023-08-

In [ ]:
# read data
data = pd.read_pickle("data.pkl")
assert data.donor_id.nunique() == 38, "Wrong number of donors"

# Testing

## Test 1: CommonBrain

In [ ]:
# define features
features = []
keys = ["_mean", "frac", "gini", "bias"]
for c in data.columns:
    if ("_score" in c) or ("_length" in c):
        if "_normed" not in c:
            continue
    for k in keys:
        if k in c:
            features.append(c)
features.append("rpm")
print("Features:", features)

In [ ]:
# define the classifier
clf = AutoML(
    task="classification",
    estimator_list=["xgboost"],
    early_stop=True,
    eval_method="cv",
    time_budget=120,  # time budget in seconds
    metric="ap",
    verbose=4,
    use_ray=True,
    split_type=StratifiedGroupKFold(n_splits=5),
)

In [ ]:
# fit
clf.fit(
    data[data.donor_id == "CommonBrain"][features],
    data[data.donor_id == "CommonBrain"]["xtea_1kb_3end"],
    groups=data[data.donor_id == "CommonBrain"]["Chromosome"],
)

In [ ]:
ddata = data[data.donor_id == "CommonBrain"]
mdl = Model(
    clf=clf,
    data=ddata,
    features=features,
    label_col="xtea_1kb_3end",
    rpm_filter=5,
    outfile=f"model_logs/CommonBrain_long.log",
)
mdl.cv(n_splits=5)